<a href="https://colab.research.google.com/github/abhinavtk1/CS6910-A2/blob/main/main_part_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MA23M002 - ABHINAV T K
# CS6910 - Assignment 2

**Github repo:** https://github.com/abhinavtk1/CS6910-A2 <br>
**Wandb report:**

In [ ]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train = 'inaturalist_12K/train/'
test = 'inaturalist_12K/val/'


In [3]:
classes = ['Amphibia','Animalia','Arachnida','Aves','Fungi',
           'Insecta','Mammalia','Mollusca','Plantae','Reptilia']


In [ ]:
# visualization
fig = plt.figure(figsize=(18,9))
r, c = 2, 5
l = 1

for item in classes:
  loc = os.path.join(test, item)
  for img in os.listdir(loc):
    img_loc=(os.path.join(loc,img))
    img = mpimg.imread(img_loc)
    img = cv2.resize(img,(128,128))
    fig.add_subplot(r, c, l)
    plt.imshow(img)
    plt.title(item)
    l+=1
    break

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
#from tensorflow.keras import layers,models
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
import wandb
import numpy as np
from types import SimpleNamespace
import random

# Part A: Training from scratch


Question 1

Define model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

In [14]:
# Reference: https://machinelearningmastery.com/building-a-convolutional-neural-network-in-pytorch/

class CNN(nn.Module):
    def __init__(self, num_filters = [32,32,64,64,128], kernel_size = [3]*5,
                 act_fn = 'relu', num_neurons = 512):
        super().__init__()

        #Conv layer 1
        self.conv1 = nn.Conv2d(3, num_filters[0],
                               kernel_size=(kernel_size[0],kernel_size[0]),
                               stride=1, padding='same')
        #x = (128 - kernel_size[0] + 2)/1 + 1
        x = 128
        if act_fn=='relu':
          self.act1 = nn.ReLU()
        elif act_fn=='tanh':
          self.act1 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act1 = nn.Sigmoid()
        self.pool1 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 2
        self.conv2 = nn.Conv2d(num_filters[0], num_filters[1],
                               kernel_size=(kernel_size[1],kernel_size[1]),
                               stride=1, padding='same')
        #x = (x - kernel_size[1] + 2)/1 + 1

        if act_fn=='relu':
          self.act2 = nn.ReLU()
        elif act_fn=='tanh':
          self.act2 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act2 = nn.Sigmoid()
        self.pool2 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 3
        self.conv3 = nn.Conv2d(num_filters[1], num_filters[2],
                               kernel_size=(kernel_size[2],kernel_size[2]),
                               stride=1, padding='same')

        if act_fn=='relu':
          self.act3 = nn.ReLU()
        elif act_fn=='tanh':
          self.act3 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act3 = nn.Sigmoid()
        self.pool3 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 4
        self.conv4 = nn.Conv2d(num_filters[2], num_filters[3],
                               kernel_size=(kernel_size[3],kernel_size[3]),
                               stride=1, padding='same')

        if act_fn=='relu':
          self.act4 = nn.ReLU()
        elif act_fn=='tanh':
          self.act4 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act4 = nn.Sigmoid()
        self.pool4 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 5
        self.conv5 = nn.Conv2d(num_filters[3], num_filters[4],
                               kernel_size=(kernel_size[4],kernel_size[4]),
                               stride=1, padding='same')

        if act_fn=='relu':
          self.act5 = nn.ReLU()
        elif act_fn=='tanh':
          self.act5 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act5 = nn.Sigmoid()
        self.pool5 = nn.MaxPool2d(2,2)
        x = x//2
        x = x*x*num_filters[4]
        print(x)

        # Flatten
        self.flatten = nn.Flatten()

        # dense layer
        self.fc6 = nn.Linear(x, num_neurons)
        if act_fn=='relu':
          self.act6 = nn.ReLU()
        elif act_fn=='tanh':
          self.act6 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act6 = nn.Sigmoid()

        # output layer
        self.fc7 = nn.Linear(num_neurons, 10)


    def forward(self, x):
        # input 3x128x128, output 32x128x128
        x = self.act1(self.conv1(x))
        # input 32x128x128, output 32x64x64
        x = self.pool1(x)

        # input 32x64x64, output 32x64x64
        x = self.act2(self.conv2(x))
        # input 32x64x64, output 32x32x32
        x = self.pool2(x)

        # input 32x32x32, output 64x32x32
        x = self.act3(self.conv3(x))
        # input 64x32x32, output 64x16x16
        x = self.pool3(x)

        # input 64x16x16, output 64x16x16
        x = self.act4(self.conv4(x))
        # input 64x16x16, output 64x8x8
        x = self.pool4(x)

        # input 64x8x8, output 128x8x8
        x = self.act5(self.conv5(x))
        # input 128x8x8, output 128x4x4
        x = self.pool5(x)

        # input 128,4,4 output 2048
        x = self.flatten(x)

        # input 2048, output 512
        x = self.act6(self.fc6(x))

        # input 512, output 10
        x = self.fc7(x)
        return x

    def train(self, model, loss_fn, optimizer, dataloader_train, dataloader_val, n_epochs = 10):
        for epoch in range(n_epochs):
            for inputs, labels in dataloader_train:
                # forward, backward, and then weight update
                inputs = inputs.to(device)
                labels = labels.to(device)
                # Forward pass
                y_pred = model(inputs)
                loss = loss_fn(y_pred, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            val_accuracy = 0
            count = 0
            for inputs, labels in dataloader_val:
                inputs = inputs.to(device)
                labels = labels.to(device)
                y_pred = model(inputs)
                val_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
                count += len(labels)
            val_accuracy /= count
            print("Epoch %d: model accuracy %.2f%%" % (epoch, val_accuracy*100))
            wandb.log({ 'epoch': epoch, 'validation_accuracy': val_accuracy * 100})

In [5]:
# Loading data
import torch
from torchvision import transforms, datasets
from torch.utils.data import random_split

# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/CS6910-A2/train'
test_dir = '/content/drive/MyDrive/CS6910-A2/val'
IMG_SIZE = (128,128)
batch_size = 32

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])           # Convert images to PyTorch tensors
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_dir, transform=transform)
val_size = int(0.1 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])

testset = datasets.ImageFolder(root=test_dir, transform=transform)

# DataLoader for the dataset
dataloader_train = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
dataloader_val = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=True)


Mounted at /content/drive


In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
!nvidia-smi

Mon Apr  1 06:12:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Q2. Training the inaturalist dataset

In [15]:
# sweep config file
sweep_config = {
    'method': 'random',
    'name' : 'sweep - demo4',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'filters_num':{
            'values': [[32]*5,[32,32,64,64,128],[32,64,64,128,128],[128]*5]
        },
        'act_fn': {
            'values': ['relu']#, 'gelu', 'silu', 'mish', 'tanh'] # nn.ReLU nn.GELU nn.SiLU nn.Mish nn.Tanh
        },
        'data_augmentation': {          # add
            'values': ['yes','no']
        },
        'batch_normalisation':{       # add
            'values': ['yes','no']
        },
        'dropout': {                  # add
            'values': [0, 0.2, 0.3]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {               # add
            'values': [32, 64, 128]
        },
        'kernel_size':{
            'values': [[3]*5, [3, 5, 5, 7, 7], [5]*5, [7]*7, [7, 5, 5, 3, 3] ]
        },
        'num_neurons_dense':{
            'values': [128, 256, 512, 1028]
        },
        'epochs':{
            'values': [10, 20, 30]
        }
    }
}

In [16]:
# Create a sweep
sweep_id = wandb.sweep(sweep = sweep_config, entity="abhinavtk", project='a2_demo')

# Export data to wandb
#wandb.log({ 'epoch': epoch, 'loss': cost, 'accuracy': accuracy * 100})
#wandb.log({ 'epoch': epoch, 'validation_loss': val_cost, 'validation_accuracy': val_accuracy * 100})


Create sweep with ID: v47bu3jc
Sweep URL: https://wandb.ai/abhinavtk/a2_demo/sweeps/v47bu3jc


In [ ]:
def main():
  with wandb.init() as run:
    run_name = "-#fil_"+str(wandb.config.filters_num)+"-ac_fn_"+wandb.config.act_fn+"-d_aug_"\
                  +wandb.config.data_augmentation + "-norm_"+wandb.config.batch_normalisation + "-bs_"\
                  +str(wandb.config.batch_size) + "-lr_"+str(wandb.config.learning_rate)+\
                  "-ep_"+str(wandb.config.epochs)

    wandb.run.name = run_name

    model = CNN(num_filters = wandb.config.filters_num, kernel_size = wandb.config.kernel_size,
                 act_fn = wandb.config.act_fn, num_neurons = wandb.config.num_neurons_dense).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate, momentum=0.9)
    model.train(model, loss_fn, optimizer, dataloader_train, dataloader_val, n_epochs = wandb.config.epochs)
wandb.agent(sweep_id, function = main, count = 10) # calls main function for count number of times
wandb.finish()

wandb: Agent Starting Run: 0vnawdss with config:
wandb: 	act_fn: relu
wandb: 	batch_normalisation: no
wandb: 	batch_size: 32
wandb: 	data_augmentation: yes
wandb: 	dropout: 0
wandb: 	epochs: 20
wandb: 	filters_num: [32, 64, 64, 128, 128]
wandb: 	kernel_size: [7, 7, 7, 7, 7, 7, 7]
wandb: 	learning_rate: 0.001
wandb: 	num_neurons_dense: 128


2048
